# Análise Exploratória dos Dados

### Importação das Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import autocorrelation_plot
import mplfinance as mpf
from app.settings import Settings

### Importação da Base de Dados

In [2]:
settings = Settings()
data = pd.read_csv(settings.DOWNLOAD_PATH+'/'+settings.NEW_NASDAQ_FILE)

### Tratamento das Variáveis

In [3]:
# Remover o símbolo '$' e substituir ',' por '.' nas colunas de preços, convertendo-as para float
columns_to_convert = ['Close/Last', 'Open', 'High', 'Low']

for col in columns_to_convert:
    data[col] = data[col].replace('[\$,]', '', regex=True).str.replace(',', '.').astype(float)

# Conversão da coluna 'Date' para datetime
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Verificar se a conversão foi bem-sucedida
print(data[columns_to_convert].dtypes)
print(data.head())

Close/Last    float64
Open          float64
High          float64
Low           float64
dtype: object
        Date  Close/Last    Volume    Open      High     Low
0 2024-11-12      208.91  38942920  208.37  209.5400  206.01
1 2024-11-11      206.84  35456010  208.50  209.6500  205.59
2 2024-11-08      208.18  36075850  209.72  209.9633  207.44
3 2024-11-07      210.05  52878380  207.44  212.2500  207.19
4 2024-11-06      207.09  72292170  200.01  207.5500  199.14


<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
C:\Users\biel_\AppData\Local\Temp\ipykernel_11388\1998124349.py:5: SyntaxWarning: invalid escape sequence '\$'
  data[col] = data[col].replace('[\$,]', '', regex=True).str.replace(',', '.').astype(float)


### Análise das Informações

In [ ]:
# Exibir informações iniciais sobre os dados
print(data.info())

In [140]:
# Ordenar por data para análises temporais
data = data.sort_values(by='Date')

In [ ]:
# Exibir informações iniciais sobre os dados
print(data.head())

In [ ]:
# Estatísticas Descritivas Básicas
print("\nEstatísticas Descritivas Básicas:")
print(data.describe())

### Análise Estátisca da Base

In [ ]:
# Médias, Medianas e Modas
print("\nMédia dos valores de fechamento:")
print(data['Close/Last'].mean())

In [ ]:
print("\nMediana dos valores de fechamento:")
print(data['Close/Last'].median())

In [ ]:
print("\nModa dos valores de fechamento:")
print(data['Close/Last'].mode()[0])

In [ ]:
# Verificar se os dados estão balanceados
print("\nValores ausentes em cada coluna:")
print(data.isnull().sum())

In [ ]:
# Estatísticas de esparcividade
sparsity = (data.isnull().sum().sum() / data.size) * 100
print(f"\nPercentual de esparcividade: {sparsity:.2f}%")

In [ ]:
#Analisar volumes e variações - disperção intrad
candle = data.loc[:,['Date','Close/Last', 'Open', 'High', 'Low','Volume']].rename(columns={'Close/Last':'Close'})
candle = candle.set_index('Date')

# Plotando o gráfico de candlesticks
mpf.plot(candle.loc[candle.index>'2024-08-31',:], type='candle', style='charles', title='Gráfico de Candlesticks', volume=True)

In [ ]:
# Visualização da Distribuição dos Dados sobre valores de fechamento
plt.figure(figsize=(12, 6))
sns.histplot(data['Close/Last'], bins=50, kde=True)
plt.title('Distribuição dos Valores de Fechamento')
plt.xlabel('Fechamento')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Análise de Correlação - Analise que as variáveis estão extremamente correlacionadas e com vazamento de dados. 
correlation_matrix = data[['Close/Last', 'Open', 'High', 'Low', 'Volume']].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
# Média móvel de 30 dias do fechamento
data['Close_Moving_Avg_30'] = data['Close/Last'].rolling(window=30).mean()

plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['Close/Last'], label='Fechamento')
plt.plot(data['Date'], data['Close_Moving_Avg_30'], label='Média Móvel 30 dias', color='orange')
plt.title('Valor de Fechamento com Média Móvel de 30 Dias')
plt.xlabel('Data')
plt.ylabel('Valor de Fechamento')
plt.legend()
plt.show()

### Análise de Retornos Diários

In [ ]:
#Cálculo do retorno diário percentual da ação (métrica importante para avaliar a volatilidade do ativo)

data['Daily_Return'] = data['Close/Last'].pct_change() * 100
plt.figure(figsize=(12, 6))
sns.histplot(data['Daily_Return'].dropna(), bins=50, kde=True)
plt.title('Distribuição dos Retornos Diários (%)')
plt.xlabel('Retorno Diário (%)')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Análise de sazonalidade por mês e ano
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

plt.figure(figsize=(10, 6))
sns.boxplot(x='Month', y='Daily_Return', data=data)
plt.title('Distribuição Mensal dos Valores de Retorno')
plt.xlabel('Mês')
plt.ylabel('Retorno')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Year', y='Daily_Return', data=data)
plt.title('Distribuição Anual dos Valores de Retorno')
plt.xlabel('Ano')
plt.ylabel('Retorno')
plt.show()

#2022 É o único ano que apresenta uma variabilidade maior com um disperção maior do primeiro e terceiro quartis

### Análise de Volatilidade

In [ ]:
#Cálculo da volatilidade dos preços de fechamento em janelas móveis (ex: 30 dias) para verificar variações ao longo do tempo.
#Quanto de volatilidade estou tendo? Desvio padrão do retorno (risco da ação)

data['Volatilidade_30d'] = data['Daily_Return'].rolling(window=30).std()
plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['Volatilidade_30d'], color='red')
plt.title('Volatilidade de 30 dias dos Retornos Diários')
plt.xlabel('Data')
plt.ylabel('Volatilidade (%)')
plt.show()

In [ ]:
#Volume Médio por Mês
#Verificação da média do volume de ações negociadas por mês, o que pode indicar períodos de maior interesse no ativo.

monthly_volume = data.groupby(data['Date'].dt.to_period("M"))['Volume'].mean()
plt.figure(figsize=(12, 6))
monthly_volume.plot(kind='bar', color='skyblue')
plt.title('Volume Médio Mensal')
plt.xlabel('Mês')
plt.ylabel('Volume Médio')
plt.xticks(rotation=45)
plt.show()

#A compra e venda parece bem estável, com apenas alguns picos. 

In [ ]:
#Análise de Correlação Temporal com Lag (Atraso)
#Análise da autocorrelação dos preços de fechamento para ver se há algum padrão temporal que se repete.

plt.figure(figsize=(12, 6))
autocorrelation_plot(data['Close/Last'].dropna())
plt.title('Autocorrelação do Preço de Fechamento')
plt.show()

In [ ]:
#Análise de Correlação Temporal com Lag (Atraso)
#Analise a autocorrelação dos preços de fechamento para ver se há algum padrão temporal que se repete.

plt.figure(figsize=(12, 6))
autocorrelation_plot(data['Daily_Return'].dropna())
plt.title('Autocorrelação do retorno diário')
plt.show()

#Variação sobre o retorno da ação % percentual de variação diária, não tem muita correlação com retorno. 
#Isso em finanças pode ser chamado de passeio aleatório que é quando o retorno da ação amanhã não pode ser explicado pelos retornos dos dias anteriores.  

In [158]:
data['forca_relativa'] = (data['Close/Last'] - data['Low'])/ (data['High'] - data['Low'])

In [ ]:
# Análise de como o fechamento se compara com a maxima e minima do dia 
# 0 - Minima 
# 1 - Maxima

plt.figure(figsize=(12, 6))
sns.histplot(data['forca_relativa'].dropna(), bins=50, kde=False)
plt.title('Distribuição dos força relativa ')
plt.xlabel('forca_relativa')
plt.ylabel('Frequência')
plt.show()

# Não há uma tendência clara do fechamento em relação a máximo e mínima 

In [ ]:
data['forca_relativa'].describe()

## Gerando a base com variáveis necessárias para Modelo

In [ ]:
# Garantir que a coluna 'Date' está no formato datetime
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Remover linhas com datas inválidas
data.dropna(subset=['Date'], inplace=True)

# Tratamento de colunas financeiras: Remover '$' e converter para float
for col in ['Close/Last', 'Open', 'High', 'Low']:
    data[col] = data[col].replace({'\$': ''}, regex=True).astype(float)

# Converter 'Volume' para float, ignorando possíveis erros
data['Volume'] = pd.to_numeric(data['Volume'], errors='coerce')

# Remover valores nulos após o cálculo da variável alvo
data.dropna(inplace=True)

# Criar features adicionais
data = data.sort_values(by = 'Date', ascending = True)
data['SMA_20'] = data['Close/Last'].rolling(window=20).mean()
data['SMA_50'] = data['Close/Last'].rolling(window=50).mean()
data['Volatility_30'] = data['Close/Last'].rolling(window=30).std()
data['Avg_Volume_30'] = data['Volume'].rolling(window=30).mean()
data = data.sort_values(by = 'Date', ascending= False)

# Remover valores nulos gerados pelas janelas móveis
data.dropna(inplace=True)

#Gerando a base para modelo
data.to_csv('base_tratada.csv', index=False)